In [2]:
import numpy as np
import numpy as np
from numpy import linalg as la
from sklearn import preprocessing as pre
import random
import math
from numpy import ma

In [3]:
#X1(CENTRIST FEATURE):
X11=np.load("F:\lab\code\MLAN\Caltech101(part)\dollar_bill\CENTRIST_feature.npy")
X12=np.load("F:\lab\code\MLAN\Caltech101(part)\Faces\CENTRIST_feature.npy")
X13=np.load("F:\lab\code\MLAN\Caltech101(part)\garfield\CENTRIST_feature.npy")
X14=np.load("F:\lab\code\MLAN\Caltech101(part)\Motorbikes\CENTRIST_feature.npy")
X15=np.load("F:\lab\code\MLAN\Caltech101(part)\snoopy\CENTRIST_feature.npy")
X16=np.load("F:\lab\code\MLAN\Caltech101(part)\stop_sign\CENTRIST_feature.npy")
X17=np.load("F:\lab\code\MLAN\Caltech101(part)\windsor_chair\CENTRIST_feature.npy")
X1=ma.row_stack((X11,X12,X13,X14,X15,X16,X17))
X1.shape

(1474, 254)

In [4]:
#X2(HOG FEATURE):
X21=np.load("F:\lab\code\MLAN\Caltech101(part)\dollar_bill\\HOG_feature.npy")
X22=np.load("F:\lab\code\MLAN\Caltech101(part)\Faces\\HOG_feature.npy")
X23=np.load("F:\lab\code\MLAN\Caltech101(part)\garfield\\HOG_feature.npy")
X24=np.load("F:\lab\code\MLAN\Caltech101(part)\Motorbikes\HOG_feature.npy")
X25=np.load("F:\lab\code\MLAN\Caltech101(part)\snoopy\HOG_feature.npy")
X26=np.load("F:\lab\code\MLAN\Caltech101(part)\stop_sign\HOG_feature.npy")
X27=np.load("F:\lab\code\MLAN\Caltech101(part)\windsor_chair\HOG_feature.npy")
X2=ma.row_stack((X21,X22,X23,X24,X25,X26,X27))
X2.shape

(1474, 1984)

In [5]:
#X3(LBP FEATURE):
X31=np.load("F:\lab\code\MLAN\Caltech101(part)\dollar_bill\\LBP_feature.npy")
X32=np.load("F:\lab\code\MLAN\Caltech101(part)\Faces\\LBP_feature.npy")
X33=np.load("F:\lab\code\MLAN\Caltech101(part)\garfield\\LBP_feature.npy")
X34=np.load("F:\lab\code\MLAN\Caltech101(part)\Motorbikes\LBP_feature.npy")
X35=np.load("F:\lab\code\MLAN\Caltech101(part)\snoopy\LBP_feature.npy")
X36=np.load("F:\lab\code\MLAN\Caltech101(part)\stop_sign\LBP_feature.npy")
X37=np.load("F:\lab\code\MLAN\Caltech101(part)\windsor_chair\LBP_feature.npy")
X3=ma.row_stack((X31,X32,X33,X34,X35,X36,X37))

In [6]:
X=[X1,X2,X3]

In [7]:
def cluster(X,c,num_of_adaptive_neighbours=9):
    
    '''
    X=[X1,X2,...,Xv]  --type:list
    '''

    #Standardlize:
    n,_=X[0].shape
    v=len(X)#v:number of different views
    for i in range(v):
        X[i]=pre.scale(X[i],axis=1)#You need to import sklearn.preprocessing as pre before using the standardlization module.
    print("Preprocession finished...")
    
    #Initialize:
    lambda_=random.randint(1,30)
    W=np.ones((v),dtype=np.float32)/v#W records the weight of each view
    alpha=np.zeros((n),dtype=np.float32)
    S=np.zeros((n,n),dtype=np.float32)#S denotes the similarity matrix
    Dist=np.zeros((n,n),dtype=np.float32)#Dist denotes the distance matrix
    F=np.zeros((n,c),dtype=np.float32)#F denotes the indicator matrix
    k_=num_of_adaptive_neighbours
    for i in range(n):
        for j in range(n):
            for k in range(v):
                Dist[i][j]+=W[k]*(la.norm(X[k][i,:]-X[k][j,:]))**2
        ascending_order_index=np.argsort(Dist[i])[1:k_+1]
        dist_of_neighbours_sorted_ascending=Dist[i][ascending_order_index]
        alpha[i]=0.5*(k_*dist_of_neighbours_sorted_ascending[k_-1]-np.sum(dist_of_neighbours_sorted_ascending))
        S[i][ascending_order_index]=(np.mean(dist_of_neighbours_sorted_ascending))/alpha[i]*0.5+1/k_
        S[i][ascending_order_index]=S[i][ascending_order_index]-dist_of_neighbours_sorted_ascending/alpha[i]*0.5
    mean_alpha=np.mean(alpha)
    #？？？论文中出现的右下角的加号是max(0,x)的意思吗？
    for i in range(n):
        for j in range(n):
            if S[i][j]<0:
                S[i][j]=0
                
    result=0
    for k in range(v):
        a=0
        for i in range(n):
            for j in range(n):
                a+=S[i][j]*(la.norm(X[k][i]-X[k][j]))**2
        result+=a**0.5
    result+=mean_alpha*(la.norm(S))**2
    print('After initialization, the objective value is %f.'%result)
    
    #Update:
    for iter_num in range(30):
        #Update W_v:
        for k in range(v):
            tmp=0
            for i in range(n):
                for j in range(n):
                    tmp+=S[i][j]*((la.norm(X[k][i]-X[k][j]))**2)
            W[k]=0.5/(tmp**0.5)
        #Update F:
        diagonal_element=(np.sum(S,axis=1)+np.sum(S.T,axis=1))*0.5
        D_s=np.zeros((n,n),dtype=np.float32)#D_s denotes the degree matrix
        for i in range(n):
            D_s[i][i]=diagonal_element[i]
        L_s=D_s-(S+S.T)/2 #L_s denotes Laplacian matrix
        eigenvalue,eigenvector=la.eig(L_s)
        sorted_indices = np.argsort(eigenvalue)
        F=eigenvector[:,sorted_indices[:c]]
        #Update S:
        S=np.zeros((n,n),dtype=np.float32)
        for i in range(n):
            for j in range(n):
                Dist[i][j]=0
                Dist[i][j]+=lambda_*la.norm(F[i,:]-F[j,:])**2 
                for k in range(v):
                    Dist[i][j]+=W[k]*(la.norm(X[k][i,:]-X[k][j,:]))**2
            ascending_order_index=np.argsort(Dist[i])[1:k_+1]
            dist_of_neighbours_sorted_ascending=Dist[i][ascending_order_index]
            alpha[i]=0.5*(k_*dist_of_neighbours_sorted_ascending[k_-1]-np.sum(dist_of_neighbours_sorted_ascending))
            S[i][ascending_order_index]=(np.mean(dist_of_neighbours_sorted_ascending))/alpha[i]*0.5+1/k_
            S[i][ascending_order_index]=S[i][ascending_order_index]-dist_of_neighbours_sorted_ascending/alpha[i]*0.5
        mean_alpha=np.mean(alpha)
        for i in range(n):
            for j in range(n):
                if S[i][j]<0:
                    S[i][j]=0
        
        #Compute the objective value:
        result=0
        for k in range(v):
            a=0
            for i in range(n):
                for j in range(n):
                    a+=S[i][j]*(la.norm(X[k][i]-X[k][j]))**2
            result+=a**0.5
        result+=mean_alpha*(la.norm(S))**2
        print('After iteration %i,'%(iter_num+1),end='')
        print(' the objective value is %f.'%result)
        
    return S
                    

In [8]:
S=cluster(X,3)

Preprocession finished...
After initialization, the objective value is 71162.152152.
After iteration 1, the objective value is 3182.490094.
After iteration 2, the objective value is 3183.188041.
After iteration 3, the objective value is 3183.401892.
After iteration 4, the objective value is 3183.463227.
After iteration 5, the objective value is 3183.474802.
After iteration 6, the objective value is 3183.462608.
After iteration 7, the objective value is 3183.437907.
After iteration 8, the objective value is 3183.439910.
After iteration 9, the objective value is 3183.447174.
After iteration 10, the objective value is 3183.459992.
After iteration 11, the objective value is 3183.465535.
After iteration 12, the objective value is 3183.463286.
After iteration 13, the objective value is 3183.464235.
After iteration 14, the objective value is 3183.464135.
After iteration 15, the objective value is 3183.464759.
After iteration 16, the objective value is 3183.464304.
After iteration 17, the obje

In [16]:
S.shape

(1474, 1474)

In [ ]:
num=0
for i in range(52):
    for j in range(52):
        if S[i][j]==0:
            num+=1
print(num)